In [3]:
from tensorflow.keras.applications import InceptionResNetV2
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from mtcnn import MTCNN
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from PIL import Image
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from deepface import DeepFace
import random
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [4]:
input_shape = (160, 160, 3)

### Custom Layers

In [5]:
class L2NormalizeLayer(layers.Layer):
    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=1)

class AbsoluteDifferenceLayer(layers.Layer):
    def call(self, inputs):
        return tf.abs(inputs[0] - inputs[1])

### Base Model

In [6]:
base_model = InceptionResNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=input_shape
)

for layer in base_model.layers[-10:]: 
    layer.trainable = True

inputs = layers.Input(shape=input_shape)
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = L2NormalizeLayer()(x)

model = models.Model(inputs, x, name="base_model")

2024-12-19 02:42:15.654492: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### Build Siamese Network

In [7]:
input_a = layers.Input(shape=input_shape)
input_b = layers.Input(shape=input_shape)

base_network = model

embedding_a = base_network(input_a)
embedding_b = base_network(input_b)

distance = AbsoluteDifferenceLayer()([embedding_a, embedding_b])

output = layers.Dense(1, activation='sigmoid')(distance)

siamese_model = models.Model(inputs=[input_a, input_b], outputs=output)

In [8]:
optimizer = optimizers.Adam(learning_rate=1e-4)

siamese_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=optimizer,
    metrics=['accuracy']
)

In [9]:
siamese_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ base_model          │ (None, 128)       │ 54,533,472 │ input_layer_2[0]… │
│ (Functional)        │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ absolute_differenc… │ (None, 128)       │          0 │ base_model[0][0], │
│ (AbsoluteDifferenc… │                   │            │ base_model[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        129 │ absolute_differe… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 54,533,601 (208.03 MB)

 Trainable params: 54,473,057 (207.80 MB)

 Non-trainable params: 60,544 (236.50 KB)

### Load Image From Directory

In [10]:
def load_images_from_directory(directory, target_size=(160, 160)):
    '''Load all images and labels from a directory.'''
    images = []
    labels = []
    label_map = {}

    for label_idx, subdir in enumerate(os.listdir(directory)):
        subdir_path = os.path.join(directory, subdir)
        if os.path.isdir(subdir_path):
            label_map[label_idx] = subdir
            for file in os.listdir(subdir_path):
                img_path = os.path.join(subdir_path, file)
                img = image.load_img(img_path, target_size=target_size)
                img_array = image.img_to_array(img) / 255.0
                images.append(img_array)
                labels.append(label_idx)
    return np.array(images), np.array(labels)

In [11]:
def create_pairs(images, labels):
    '''Create positive and negative pairs from images and labels.'''
    pairs = []
    targets = []

    # Group images by labels
    label_to_images = {}
    for img, label in zip(images, labels):
        if label not in label_to_images:
            label_to_images[label] = []
        label_to_images[label].append(img)

    # Create positive and negative pairs
    for label in label_to_images:
        same_class_images = label_to_images[label]
        for i in range(len(same_class_images)):
            # Positive pair
            img1 = same_class_images[i]
            img2 = random.choice(same_class_images)
            pairs.append([img1, img2])
            targets.append(1)  # Positive label

            # Negative pair
            other_label = random.choice([l for l in label_to_images if l != label])
            img3 = random.choice(label_to_images[other_label])
            pairs.append([img1, img3])
            targets.append(0)  # Negative label

    return np.array(pairs), np.array(targets)

In [15]:
# Load training and validation data
train_images, train_labels = load_images_from_directory('dataset/train')
val_images, val_labels = load_images_from_directory('dataset/val')

# Create pairs for training and validation
x_train_pairs, y_train = create_pairs(train_images, train_labels)
x_val_pairs, y_val = create_pairs(val_images, val_labels)

# Split pairs into two inputs
x_train_1, x_train_2 = x_train_pairs[:, 0], x_train_pairs[:, 1]
x_val_1, x_val_2 = x_val_pairs[:, 0], x_val_pairs[:, 1]

In [17]:
siamese_model.fit(
    [x_train_1, x_train_2], y_train,
    validation_data=([x_val_1, x_val_2], y_val),
    batch_size=32,
    epochs=20
)

Epoch 1/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 553s 5s/step - accuracy: 0.5212 - loss: 0.6897 - val_accuracy: 0.6694 - val_loss: 0.6437
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 409s 5s/step - accuracy: 0.7428 - loss: 0.6072 - val_accuracy: 0.7153 - val_loss: 0.6059
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 405s 5s/step - accuracy: 0.8013 - loss: 0.5679 - val_accuracy: 0.7514 - val_loss: 0.5871
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 404s 4s/step - accuracy: 0.8229 - loss: 0.5484 - val_accuracy: 0.7542 - val_loss: 0.5856
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 437s 5s/step - accuracy: 0.8622 - loss: 0.5295 - val_accuracy: 0.7792 - val_loss: 0.5745
Epoch 6/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 404s 4s/step - accuracy: 0.8998 - loss: 0.5028 - val_accuracy: 0.7806 - val_loss: 0.5711
Epoch 7/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 408s 5s/step - accuracy: 0.9344 - loss: 0.4870 - val_accuracy: 0.8000 - val_loss: 0.5657
Epoch 8/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 405s 5s/step - accuracy: 0.9451 - loss: 0.4698 - val_accuracy: 0.7889 - v

2024-12-19 05:00:45.971606: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 19661056 bytes after encountering the first element of size 19661056 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


90/90 ━━━━━━━━━━━━━━━━━━━━ 416s 5s/step - accuracy: 0.9895 - loss: 0.3096 - val_accuracy: 0.8097 - val_loss: 0.5183


In [18]:
loss, acc = siamese_model.evaluate([x_val_1, x_val_2], y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {acc}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.8153 - loss: 0.5145
Validation Loss: 0.5182710289955139, Validation Accuracy: 0.8097222447395325


In [19]:
detector = MTCNN()

def preprocess_image_with_mtcnn(img_path, target_size=(160, 160)):
    '''Preprocess an image: detect face with MTCNN, crop, and resize.'''
    # Load image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB (MTCNN expects RGB format)
    
    # Detect faces
    detections = detector.detect_faces(img)
    
    if detections:
        # If a face is detected, crop the face region
        x, y, width, height = detections[0]['box']  # Get bounding box of the first detected face
        x, y = max(0, x), max(0, y)  # Ensure bounding box is within image bounds
        cropped_face = img[y:y+height, x:x+width]
    else:
        # If no face is detected, use the full image
        cropped_face = img
    
    # Resize cropped face or full image to target size
    resized_img = cv2.resize(cropped_face, target_size)
    
    # Normalize pixel values and add batch dimension
    img_array = resized_img / 255.0  # Normalize to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

In [23]:
img_path_1 = 'dataset_copy/train/Achmad Raihan/IMG_20241117_161935.jpg'
img_path_2 = 'jokowi_1.jpeg'

img1 = preprocess_image_with_mtcnn(img_path_1)
img2 = preprocess_image_with_mtcnn(img_path_2)

# Prediksi dengan Siamese Network
similarity_score = siamese_model.predict([img1, img2])

print("Similarity Score:", similarity_score[0][0])

# Interpretasi Hasil
if similarity_score[0][0] > 0.5:
    print("Hasil: Gambar kemungkinan wajah dari orang yang SAMA.")
else:
    print("Hasil: Gambar kemungkinan wajah dari orang yang BERBEDA.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
Similarity Score: 0.4094489
Hasil: Gambar kemungkinan wajah dari orang yang BERBEDA.


In [21]:
siamese_model.save('siamese_baru_lagi.keras')